In [0]:
%run "../includes/configurations"

In [0]:
from pyspark.sql.functions import to_timestamp, to_date

In [0]:
from pyspark.sql.functions import from_unixtime, unix_timestamp

In [0]:
order_items_df = spark.read.option("header", True)\
.option("inferSchema", True)\
.csv(f"{raw_folder_path}/olist_order_items_dataset.csv")\
.withColumn("shipping_limit_date_converted", to_timestamp("shipping_limit_date"))\
.drop('shipping_limit_date')\
.withColumnRenamed("shipping_limit_date_converted", "shipping_limit_date")

In [0]:
order_items_df.printSchema()

root
-- order_id: string (nullable = true)
-- order_item_id: integer (nullable = true)
-- product_id: string (nullable = true)
-- seller_id: string (nullable = true)
-- price: double (nullable = true)
-- freight_value: double (nullable = true)
-- shipping_limit_date: timestamp (nullable = true)

In [0]:
display(order_items_df.take(10))

order_id,order_item_id,product_id,seller_id,price,freight_value,shipping_limit_date
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29,2017-09-19T09:45:35.000+0000
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.9,19.93,2017-05-03T11:05:13.000+0000
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.0,17.87,2018-01-18T14:48:30.000+0000
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,2018-08-15T10:10:18.000+0000
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.9,18.14,2017-02-13T13:57:51.000+0000
00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,21.9,12.69,2017-05-23T03:55:27.000+0000
00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,7040e82f899a04d1b434b795a43b4617,19.9,11.85,2017-12-14T12:10:31.000+0000
000576fe39319847cbb9d288c5617fa6,1,557d850972a7d6f792fd18ae1400d9b6,5996cddab893a4652a15592fb58ab8db,810.0,70.75,2018-07-10T12:30:45.000+0000
0005a1a1728c9d785b8e2b08b904576c,1,310ae3c140ff94b03219ad0adc3c778f,a416b6a846a11724393025641d4edd5e,145.95,11.65,2018-03-26T18:31:29.000+0000
0005f50442cb953dcd1d21e1fb923495,1,4535b0e1091c278dfd193e5a1d63b39f,ba143b05f0110f0dc71ad71b4466ce92,53.99,11.4,2018-07-06T14:10:56.000+0000


In [0]:
order_items_df.count()    #VALID

Out[8]: 112650

#### SELECTING REQUIRED COLUMNS TO BE WRITTEN IN PROCESSED CONTAINER

In [0]:
final_order_items_df = order_items_df.select('order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price')

In [0]:
display(final_order_items_df.take(10))

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19T09:45:35.000+0000,58.9
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03T11:05:13.000+0000,239.9
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18T14:48:30.000+0000,199.0
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15T10:10:18.000+0000,12.99
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13T13:57:51.000+0000,199.9
00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,2017-05-23T03:55:27.000+0000,21.9
00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,7040e82f899a04d1b434b795a43b4617,2017-12-14T12:10:31.000+0000,19.9
000576fe39319847cbb9d288c5617fa6,1,557d850972a7d6f792fd18ae1400d9b6,5996cddab893a4652a15592fb58ab8db,2018-07-10T12:30:45.000+0000,810.0
0005a1a1728c9d785b8e2b08b904576c,1,310ae3c140ff94b03219ad0adc3c778f,a416b6a846a11724393025641d4edd5e,2018-03-26T18:31:29.000+0000,145.95
0005f50442cb953dcd1d21e1fb923495,1,4535b0e1091c278dfd193e5a1d63b39f,ba143b05f0110f0dc71ad71b4466ce92,2018-07-06T14:10:56.000+0000,53.99


In [0]:
final_order_items_df.count()

Out[11]: 112650

In [0]:
final_order_items_df.write.mode('overwrite').parquet(f"{processed_folder_path}/order_items")

In [0]:
final_order_items_df.coalesce(1).write.mode('overwrite').option("header", "true").csv(f'{processed_folder_path}/csv/order_items_processed.csv')